In [1]:
import os
from os.path import join
import glob
import pandas as pd
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt
%load_ext autoreload

In [2]:
def display_all(df):
    with pd.option_context("display.max_rows", 100, "display.max_columns", 100): 
        print(df)

In [3]:

def make_value2index(attacks):
    #make dictionary
    attacks = sorted(attacks)
    d = {}
    counter=0
    for attack in attacks:
        d[attack] = counter
        counter+=1
    return d

In [4]:

# chganges label from string to integer/index
def encode_label(Y_str):
    labels_d = make_value2index(np.unique(Y_str))
    Y = [labels_d[y_str] for y_str  in Y_str]
    Y = np.array(Y)
    return np.array(Y)

In [5]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/Colab Notebooks/NetworkAnomalyDetectionML/")

Mounted at /content/drive


In [6]:
# All columns
col_names = np.array(['Source Port', 'Destination Port',
                      'Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets',
                      'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean',
                      'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std',
                      'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total',
                      'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
                      'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd URG Flags', 'Fwd Header Length', 'Bwd Header Length',
                      'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std',
                      'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
                      'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio', 'Average Packet Size', 'Avg Fwd Segment Size',
                      'Avg Bwd Segment Size','Subflow Fwd Packets', 'Subflow Fwd Bytes',
                      'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward',
                      'act_data_pkt_fwd', 'min_seg_size_forward', 'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean',
                      'Idle Std', 'Idle Max', 'Idle Min', 'Label'])

In [7]:
start = 11
col_names = col_names[start:]

In [8]:
col_names.shape

(61,)

In [9]:
df_train = pd.read_csv('./train_set.csv',names=col_names, skiprows=1)
df_train = df_train.replace([-np.inf, np.inf], np.nan)
df_train = df_train.dropna()

In [10]:
print(df_train.shape)

(556690, 61)


In [11]:
df_train['Label'].value_counts()

BENIGN                        278345
DoS Hulk                      115062
PortScan                       79465
DDoS                           64013
DoS GoldenEye                   5146
FTP-Patator                     3969
SSH-Patator                     2948
DoS slowloris                   2898
DoS Slowhttptest                2749
Bot                              983
Web Attack  Brute Force         753
Web Attack  XSS                 326
Infiltration                      18
Web Attack  Sql Injection        10
Heartbleed                         5
Name: Label, dtype: int64

In [12]:
df_test = pd.read_csv('./test_set.csv',names=col_names, skiprows=1)  
print('Test set size: ', df_test.shape)
df_test = df_test.replace([-np.inf, np.inf], np.nan)
df_test= df_test.dropna()

df_val = pd.read_csv('./crossval_set.csv',names=col_names, skiprows=1)  
df_val = df_val.replace([-np.inf, np.inf], np.nan)
df_val = df_val.dropna()
print('Validation set size: ', df_val.shape)

Test set size:  (278338, 61)
Validation set size:  (278338, 61)


In [13]:
df_label = df_train['Label']
data = df_train.drop(columns=['Label'])
X_train = data.values
y_train = encode_label(df_label.values)

In [14]:
from sklearn.preprocessing import MinMaxScaler

In [15]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_train

array([[6.92500003e-01, 1.49999984e-07, 6.92500000e-01, ...,
        0.00000000e+00, 6.92500000e-01, 6.92500000e-01],
       [4.58333330e-07, 5.58333273e-07, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [9.17666659e-05, 7.09166590e-06, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [8.20000001e-01, 1.33333319e-07, 8.20833333e-01, ...,
        0.00000000e+00, 8.20000000e-01, 8.20000000e-01],
       [7.66666660e-07, 8.66666573e-07, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.16666663e-08, 1.41666651e-07, 3.33333333e-08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [16]:
df_label = df_test['Label']
data = df_test.drop(columns=['Label'])
X_test = data.values
y_test = encode_label(df_label.values)

In [17]:

scaler = MinMaxScaler()
X_test = scaler.fit_transform(X_test)
X_test

array([[3.15166664e-05, 3.18823495e-05, 3.15083333e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.33333341e-01, 1.26050406e-07, 1.37500000e-01, ...,
        0.00000000e+00, 1.33333333e-01, 1.33333333e-01],
       [5.57552229e-01, 7.82646973e-04, 5.58332600e-01, ...,
        0.00000000e+00, 5.57552225e-01, 5.57552225e-01],
       ...,
       [5.83333328e-08, 1.59663848e-07, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.90833336e-01, 1.17647046e-07, 6.90833333e-01, ...,
        0.00000000e+00, 6.90833333e-01, 6.90833333e-01],
       [5.99999995e-07, 7.05882276e-07, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [18]:
df_label = df_val['Label']
data = df_val.drop(columns=['Label'])
X_val = data.values
y_val = encode_label(df_label.values)

In [19]:
scaler = MinMaxScaler()
X_val = scaler.fit_transform(X_val)
X_val

array([[2.57266665e-04, 4.91666618e-07, 4.00000000e-07, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.06815412e-02, 2.03333313e-06, 6.06877333e-02, ...,
        0.00000000e+00, 6.06815333e-02, 6.06815333e-02],
       [5.74729720e-01, 1.99166647e-04, 5.75011500e-01, ...,
        0.00000000e+00, 5.74729717e-01, 5.74729717e-01],
       ...,
       [3.93333330e-06, 4.02499960e-06, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.39374998e-04, 2.39466643e-04, 2.39366667e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.86666665e-06, 5.08333283e-07, 4.08333333e-07, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [20]:
from tensorflow.keras.utils import to_categorical

In [21]:
y_train = to_categorical(y_train, 15)
y_test = to_categorical(y_test, 15)
y_val = to_categorical(y_val, 15)

In [22]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, BatchNormalization, Flatten, Dense, Activation,Dropout
from tensorflow.keras.constraints import max_norm

In [23]:
#hyper-params
batch_size = 1024 # increasing batch size with more gpu added
input_dim = X_train.shape[1]
num_class = 15                   # 15 intrusion classes, including benign traffic class
num_epochs = 3
learning_rates = 1e-3
regularizations = 1e-3
optim = tf.keras.optimizers.Adam(lr=learning_rates, beta_1=0.9, beta_2=0.999, epsilon=1e-8)

print(input_dim)
print(num_class)

60
15


In [24]:
X_train_r = np.zeros((len(X_train), input_dim, 1))
X_train_r[:, :, 0] = X_train[:, :input_dim]
print(X_train_r.shape)

(556690, 60, 1)


In [25]:
X_test_r = np.zeros((len(X_test), input_dim, 1))
X_test_r[:, :, 0] = X_test[:, :input_dim]
print(X_test_r.shape)

(278338, 60, 1)


In [26]:
X_val_r = np.zeros((len(X_val), input_dim, 1))
X_val_r[:, :, 0] = X_val[:, :input_dim]
print(X_val_r.shape)

(278338, 60, 1)


In [27]:
model = Sequential()

# input layer
model.add(Conv1D(filters=64, kernel_size=3, padding='same', input_shape=(71-start,1)))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))


model.add(Conv1D(filters=128, kernel_size=3))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))

model.add(Flatten())
#model.add(Dense(100, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(num_class))
model.add(Activation('softmax'))


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 60, 64)            256       
_________________________________________________________________
batch_normalization (BatchNo (None, 60, 64)            240       
_________________________________________________________________
activation (Activation)      (None, 60, 64)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 58, 128)           24704     
_________________________________________________________________
batch_normalization_1 (Batch (None, 58, 128)           232       
_________________________________________________________________
activation_1 (Activation)    (None, 58, 128)           0         
_________________________________________________________________
flatten (Flatten)            (None, 7424)              0

In [28]:
model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])

In [29]:
model.fit(X_train_r, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_val_r, y_val), verbose=1)

Epoch 1/3
544/544 [==============================] - 388s 711ms/step - loss: 0.7315 - accuracy: 0.7648 - val_loss: 0.2360 - val_accuracy: 0.9140
Epoch 2/3
544/544 [==============================] - 386s 709ms/step - loss: 0.1270 - accuracy: 0.9588 - val_loss: 0.1047 - val_accuracy: 0.9678
Epoch 3/3
544/544 [==============================] - 389s 716ms/step - loss: 0.0958 - accuracy: 0.9687 - val_loss: 0.0822 - val_accuracy: 0.9745


In [30]:
accuracy = model.evaluate(X_test_r, y_test, batch_size=batch_size, verbose=0)

In [31]:
model.save('./cicids2017cnn.h5')

In [32]:
weights_list = model.get_weights()
i = 0
for item in weights_list:
    i=i+1
    print("Layer " + str(i))
    if (i == 15):
        fc = item.squeeze()

print(fc)

Layer 1
Layer 2
Layer 3
Layer 4
Layer 5
Layer 6
Layer 7
Layer 8
Layer 9
Layer 10
Layer 11
Layer 12
Layer 13
Layer 14
Layer 15
Layer 16
[[-1.68125615e-01 -3.20429981e-01 -1.44134417e-01  2.69826412e-01
   3.82113785e-01 -5.97994566e-01 -8.91191661e-02  3.74762535e-01
   7.08100647e-02 -3.76121253e-01 -9.57596526e-02  3.23393315e-01
  -8.05765092e-02 -7.50905350e-02 -8.37573037e-02]
 [ 1.86685875e-01 -1.91963002e-01 -2.52047241e-01  2.24342331e-01
  -3.70688550e-02  1.35466978e-01  2.52895504e-01 -3.89544927e-02
  -2.12894738e-01 -1.28927767e-01 -2.65423119e-01 -1.31036863e-01
   2.29641363e-01 -7.13735372e-02  1.76639214e-01]
 [ 6.32681549e-02  3.02115023e-01 -4.36747260e-02  2.34494835e-01
  -1.19117022e-01  3.87737870e-01  3.31916869e-01  1.40284136e-01
  -3.02585602e-01  3.05571169e-01  2.63624996e-01  3.71345490e-01
  -4.38185260e-02  2.99706347e-02  2.74805993e-01]
 [ 7.76343793e-02 -7.38935098e-02 -4.34885085e-01  3.37858737e-01
   7.02532604e-02 -3.90608758e-01 -2.28581324e-01 -4

In [33]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [34]:
df_label = df_test['Label']
ytest = encode_label(df_label.values)


In [35]:

# predict probabilities for test set
yhat_probs = model.predict(X_test_r, verbose=0)

# predict crisp classes for test set
yhat_classes = model.predict_classes(X_test_r, verbose=0)

# reduce to 1d array
yhat_probs = yhat_probs[:, 0]
#yhat_classes = yhat_classes[:, 0]

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [36]:
accuracy = accuracy_score(ytest, yhat_classes)
print('Accuracy: %f' % accuracy)

Accuracy: 0.970593


In [37]:
precision = precision_score(ytest, yhat_classes, labels=[1,2], average='micro')
# labels is a list of all possible class labels
print('Precision: %f' % precision)

Precision: 0.971445


In [38]:
recall = recall_score(ytest, yhat_classes, average='weighted')
print('Recall: %f' % recall)

Recall: 0.970593


In [39]:
f1 = f1_score(ytest, yhat_classes, average='weighted')
print('F1 score: %f' % f1)

F1 score: 0.969177


In [40]:
kappa = cohen_kappa_score(ytest, yhat_classes)
print('Cohens kappa: %f' % kappa)

# confusion matrix
matrix = confusion_matrix(ytest, yhat_classes)
print(matrix)

Cohens kappa: 0.956599
[[133290    186    755     70   2419     60     33    323      0      0
    2026      7      0      0      0]
 [   204    287      0      0      0      0      0      0      0      0
       0      0      0      0      0]
 [    42      0  31862      2    100      0      0      0      0      0
       0      0      0      0      0]
 [    83      0      0   2487      0      0      3      0      0      0
       0      0      0      0      0]
 [   149      0      1     15  57334      0      0      9      0      0
       0     23      0      0      0]
 [    63      0      0      2      0   1245     64      0      0      0
       0      0      0      0      0]
 [    49      0      0      2      0     27   1302     69      0      0
       0      0      0      0      0]
 [     5      0      0      0      3      0      3   1973      0      0
       0      0      0      0      0]
 [     1      0      0      0      0      0      0      0      0      0
       0      1      0   

In [41]:
!pip install shap

     |████████████████████████████████| 358kB 4.5MB/s 
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491618 sha256=77360f5b78f976ad272959a1b328e97cbd0ca9f37c520152d8b6d3b8d7582aa7
  Stored in directory: /root/.cache/pip/wheels/15/27/f5/a8ab9da52fd159aae6477b5ede6eaaec69fd130fa0fa59f283
Successfully built shap


In [42]:
import shap

In [43]:
X_train_r.shape

(556690, 60, 1)

In [44]:
import shap

In [46]:
X_train_r.shape

(556690, 60, 1)

In [45]:
background =X_train_r[np.random.choice(X_train_r.shape[0], 100, replace=False)]

# explain predictions of the model on four images
e = shap.DeepExplainer(model, background)

shap_values = e.shap_values(X_test_r[1:5])

keras is no longer supported, please use tf.keras instead.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode. See PR #1483 for discussion.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


StagingError: ignored